In [1]:
import pandas as pd
from tensorflow import keras
from os import path
import pickle


In [2]:
from proteinbert.finetuning import encode_train_and_valid_sets, encode_dataset
from proteinbert import OutputType, OutputSpec, evaluate_by_len, load_pretrained_model

In [3]:
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, \
finetune, evaluate_by_len

from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [4]:
import wandb
from wandb.keras import WandbCallback

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [6]:
DATA_DIR = "../../data/"

In [7]:
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

In [8]:
pretrained_model_generator, input_encoder = load_pretrained_model("../../data/protein_bert/", "epoch_92400_sample_23500000.pkl")

In [9]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

In [13]:
wandb.init(project=f"March_finetune", entity="kvetab")

wandb: Currently logged in as: kvetab (use `wandb login --relogin` to force relogin)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [10]:
train_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train_data.csv"), index_col=0)
valid_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_valid_data.csv"), index_col=0)
test_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test_data.csv"), index_col=0)
train_data.head()

,Antibody_ID,heavy,light,Y
2073,6aod,EVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLE...,DIVMTKSPSSLSASVGDRVTITCRASQGIRNDLGWYQQKPGKAPKR...,0
1517,4yny,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,EFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2025,5xcv,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,QFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2070,6and,EVQLVESGGGLVQPGGSLRLSCAASGYEFSRSWMNWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRSSQSIVHSVGNTFLEWYQQKPG...,1
666,2xqy,QVQLQQPGAELVKPGASVKMSCKASGYSFTSYWMNWVKQRPGRGLE...,DIVLTQSPASLALSLGQRATISCRASKSVSTSGYSYMYWYQQKPGQ...,0


In [11]:
train_data["seq"] = train_data["heavy"] + train_data["light"]
valid_data["seq"] = valid_data["heavy"] + valid_data["light"]
test_data["seq"] = test_data["heavy"] + test_data["light"]

In [14]:
training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 3, factor = 0.25, min_lr = 1e-07, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 6, restore_best_weights = True),
    WandbCallback()
]

In [12]:
epoch_num = 50
batch_size = 128
learning_rate = 1e-4

In [13]:
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }

In [14]:
finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], valid_data['seq'], valid_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)

[2022_03_07-12:38:15] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_03_07-12:38:15] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_03_07-12:38:15] Training with frozen pretrained layers...


2022-03-07 12:38:16.543610: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-07 12:38:17.659971: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9656 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:41:00.0, compute capability: 7.5
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-03-07 12:38:19.665660: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are en

Epoch 1/50


2022-03-07 12:38:28.305649: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 7605


11/11 [==============================] - 12s 327ms/step - loss: 0.7412 - val_loss: 0.5121


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/50
11/11 [==============================] - 1s 108ms/step - loss: 0.4909 - val_loss: 0.5763
Epoch 3/50
11/11 [==============================] - 1s 110ms/step - loss: 0.5073 - val_loss: 0.4869
Epoch 4/50
11/11 [==============================] - 1s 108ms/step - loss: 0.4595 - val_loss: 0.4831
Epoch 5/50
11/11 [==============================] - 1s 113ms/step - loss: 0.4504 - val_loss: 0.4515
Epoch 6/50
11/11 [==============================] - 1s 109ms/step - loss: 0.4326 - val_loss: 0.4645
Epoch 7/50
11/11 [==============================] - 1s 108ms/step - loss: 0.4454 - val_loss: 0.4529
Epoch 8/50
11/11 [==============================] - 1s 108ms/step - loss: 0.4213 - val_loss: 0.4423
Epoch 9/50
11/11 [==============================] - 1s 108ms/step - loss: 0.4188 - val_loss: 0.4585
Epoch 10/50
11/11 [==============================] - 1s 108ms/step - loss: 0.4261 - val_loss: 0.4314
Epoch 11/50
11/11 [==============================] - 1s 107ms/step - loss: 0.4088 - val_loss: 0.426

In [20]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [21]:
print('Test-set performance:')
display(results)

print('Confusion matrix:')
display(confusion_matrix)

Test-set performance:


,# records,AUC
Model seq len,,
512,119,0.946784
All,119,0.946784


Confusion matrix:


,0,1
0,89,7
1,8,15


In [23]:
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

0.6666666666666666

In [19]:
mod = model_generator.create_model(seq_len = 512)
mod.save(path.join(DATA_DIR, "protein_bert/2022_03_07__01.pkl"))

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-03-07 12:45:22.012863: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_07__01.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_07__01.pkl/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# 2

In [1]:
import tensorflow as tf
print(tf. __version__)

2.6.2


In [27]:
epoch_num = 50
batch_size = 128
learning_rate = 1e-4
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }
finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], valid_data['seq'], valid_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)

[2022_03_07-12:50:19] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_03_07-12:50:19] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_03_07-12:50:19] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/50
11/11 [==============================] - 10s 280ms/step - loss: 0.7594 - val_loss: 0.5533
Epoch 2/50
11/11 [==============================] - 1s 108ms/step - loss: 0.5586 - val_loss: 0.5529
Epoch 3/50
11/11 [==============================] - 1s 108ms/step - loss: 0.5223 - val_loss: 0.4892
Epoch 4/50
11/11 [==============================] - 1s 108ms/step - loss: 0.4937 - val_loss: 0.5385
Epoch 5/50
11/11 [==============================] - 1s 109ms/step - loss: 0.4691 - val_loss: 0.4616
Epoch 6/50
11/11 [==============================] - 1s 108ms/step - loss: 0.4576 - val_loss: 0.5050
Epoch 7/50
11/11 [==============================] - 1s 108ms/step - loss: 0.4565 - val_loss: 0.4543
Epoch 8/50
11/11 [==============================] - 1s 109ms/step - loss: 0.4226 - val_loss: 0.4396
Epoch 9/50
11/11 [==============================] - 1s 109ms/step - loss: 0.4127 - val_loss: 0.4365
Epoch 10/50
11/11 [==============================] - 1s 108ms/step - loss: 0.4285 - val_loss: 0.590

11/11 [==============================] - 10s 424ms/step - loss: 0.3950 - val_loss: 0.3896
Epoch 2/50
11/11 [==============================] - 3s 251ms/step - loss: 0.3596 - val_loss: 0.3884
Epoch 3/50
11/11 [==============================] - 3s 252ms/step - loss: 0.3535 - val_loss: 0.4016
Epoch 4/50
11/11 [==============================] - 3s 253ms/step - loss: 0.3590 - val_loss: 0.4105
Epoch 5/50
11/11 [==============================] - 3s 253ms/step - loss: 0.3456 - val_loss: 0.3785
Epoch 6/50
11/11 [==============================] - 3s 254ms/step - loss: 0.3587 - val_loss: 0.3791
Epoch 7/50
11/11 [==============================] - 3s 254ms/step - loss: 0.3341 - val_loss: 0.3717


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


Epoch 8/50
11/11 [==============================] - 3s 253ms/step - loss: 0.3297 - val_loss: 0.3746
Epoch 9/50
11/11 [==============================] - 3s 253ms/step - loss: 0.3338 - val_loss: 0.3943
Epoch 10/50
11/11 [==============================] - 3s 253ms/step - loss: 0.3164 - val_loss: 0.3641
Epoch 11/50
11/11 [==============================] - 3s 254ms/step - loss: 0.2924 - val_loss: 0.3959
Epoch 12/50
11/11 [==============================] - 3s 254ms/step - loss: 0.2997 - val_loss: 0.3778
Epoch 13/50
11/11 [==============================] - 3s 253ms/step - loss: 0.2747 - val_loss: 0.3684

Epoch 00013: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
Epoch 14/50
11/11 [==============================] - 3s 253ms/step - loss: 0.2774 - val_loss: 0.3766
Epoch 15/50
11/11 [==============================] - 3s 254ms/step - loss: 0.2629 - val_loss: 0.3693
Epoch 16/50
11/11 [==============================] - 3s 254ms/step - loss: 0.2639 - val_loss: 0.3693

Epoch 00016

21/21 [==============================] - 13s 345ms/step - loss: 0.3221 - val_loss: 0.3717


In [28]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)
print('Confusion matrix:')
display(confusion_matrix)
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

Confusion matrix:


,0,1
0,85,11
1,5,18


0.6923076923076923

In [29]:
mod = model_generator.create_model(seq_len = 512)
mod.save(path.join(DATA_DIR, "protein_bert/2022_03_07__02.pkl"))

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_07__02.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_07__02.pkl/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# 5, 6

In [46]:
wandb.init(project=f"March_finetune", entity="kvetab")

epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▁
loss,█▄▃▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
lr,████████████▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▅▄▄▄▃▅▃▄▃▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▃▁▁▁
best_epoch,17
best_val_loss,0.36364
epoch,0
loss,0.31084
lr,0.0
val_loss,0.36835


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [15]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

In [16]:
training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 4, factor = 0.25, min_lr = 1e-07, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 8, restore_best_weights = True),
    WandbCallback()
]

In [17]:
epoch_num = 100
batch_size = 128
learning_rate = 5e-5
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }
finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], valid_data['seq'], valid_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 512, final_lr = learning_rate / 10, callbacks = training_callbacks)

[2022_03_30-14:56:34] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_03_30-14:56:34] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_03_30-14:56:34] Training with frozen pretrained layers...


2022-03-30 14:56:37.927515: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-30 14:56:41.171695: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9656 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:21:00.0, compute capability: 7.5
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-03-30 14:56:43.472165: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are en

Epoch 1/100


2022-03-30 14:56:54.924980: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 7605


11/11 [==============================] - 15s 327ms/step - loss: 0.8103 - val_loss: 0.8175


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/100
11/11 [==============================] - 1s 109ms/step - loss: 0.5943 - val_loss: 0.5337
Epoch 3/100
11/11 [==============================] - 1s 108ms/step - loss: 0.5033 - val_loss: 0.4976
Epoch 4/100
11/11 [==============================] - 1s 107ms/step - loss: 0.4616 - val_loss: 0.4737
Epoch 5/100
11/11 [==============================] - 1s 115ms/step - loss: 0.4389 - val_loss: 0.4583
Epoch 6/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4436 - val_loss: 0.4796
Epoch 7/100
11/11 [==============================] - 1s 110ms/step - loss: 0.4345 - val_loss: 0.4501
Epoch 8/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4449 - val_loss: 0.4451
Epoch 9/100
11/11 [==============================] - 1s 106ms/step - loss: 0.4531 - val_loss: 0.4949
Epoch 10/100
11/11 [==============================] - 1s 106ms/step - loss: 0.4386 - val_loss: 0.4279
Epoch 11/100
11/11 [==============================] - 1s 107ms/step - loss: 0.4162 - val_l

In [18]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)
print('Confusion matrix:')
display(confusion_matrix)
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

Confusion matrix:


,0,1
0,94,2
1,11,12


0.6486486486486487

In [19]:
mod = model_generator.create_model(seq_len = 512)
mod.save(path.join(DATA_DIR, "protein_bert/2022_03_30__05.pkl"))

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-03-30 15:01:21.837698: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_30__05.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_30__05.pkl/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# 7, 8

In [58]:
wandb.init(project=f"March_finetune", entity="kvetab")

epoch,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██▁▁▂▂▂▃▃▃▃▄▁
loss,█▄▃▃▄▄▃▃▃▃▃▃▃▂▂▂▂▃▂▂▂▂▂▂▅▃▂▂▂▁▁▁▁▁▁
lr,███████████████████▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▇▅▅▆█▄▄▃▄▆▃▃▂▃▃▂▂▂▂▂▂▂▂▂▆▄▂▂▁▁▁▁▄▂▁
best_epoch,5
best_val_loss,0.37816
epoch,0
loss,0.34296
lr,5e-05
val_loss,0.37942


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [59]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

In [60]:
training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 3, factor = 0.25, min_lr = 1e-07, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 6, restore_best_weights = True),
    WandbCallback()
]

In [61]:
epoch_num = 100
batch_size = 128
learning_rate = 1e-4
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }
finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], valid_data['seq'], valid_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)

[2022_03_07-13:31:05] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_03_07-13:31:05] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_03_07-13:31:06] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
11/11 [==============================] - 9s 285ms/step - loss: 0.7924 - val_loss: 0.7399


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/100
11/11 [==============================] - 1s 108ms/step - loss: 0.5823 - val_loss: 0.5075
Epoch 3/100
11/11 [==============================] - 1s 114ms/step - loss: 0.4952 - val_loss: 0.4969
Epoch 4/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4732 - val_loss: 0.5046
Epoch 5/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4584 - val_loss: 0.4563
Epoch 6/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4429 - val_loss: 0.4501
Epoch 7/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4343 - val_loss: 0.4438
Epoch 8/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4186 - val_loss: 0.4389
Epoch 9/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4136 - val_loss: 0.4374
Epoch 10/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4113 - val_loss: 0.4322
Epoch 11/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4250 - val_l

11/11 [==============================] - 10s 419ms/step - loss: 0.3756 - val_loss: 0.3989
Epoch 2/100
11/11 [==============================] - 3s 254ms/step - loss: 0.3735 - val_loss: 0.3986
Epoch 3/100
11/11 [==============================] - 3s 254ms/step - loss: 0.3734 - val_loss: 0.3865
Epoch 4/100
11/11 [==============================] - 3s 254ms/step - loss: 0.3646 - val_loss: 0.3801
Epoch 5/100
11/11 [==============================] - 3s 253ms/step - loss: 0.3616 - val_loss: 0.3768
Epoch 6/100
11/11 [==============================] - 3s 254ms/step - loss: 0.3611 - val_loss: 0.3848
Epoch 7/100
11/11 [==============================] - 3s 254ms/step - loss: 0.3646 - val_loss: 0.4085
Epoch 8/100
11/11 [==============================] - 3s 254ms/step - loss: 0.3483 - val_loss: 0.3732
Epoch 9/100
11/11 [==============================] - 3s 254ms/step - loss: 0.3511 - val_loss: 0.3730
Epoch 10/100
11/11 [==============================] - 3s 256ms/step - loss: 0.3188 - val_loss: 0.3716


21/21 [==============================] - 13s 344ms/step - loss: 0.3268 - val_loss: 0.3726


In [62]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)
print('Confusion matrix:')
display(confusion_matrix)
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

Confusion matrix:


,0,1
0,88,8
1,8,15


0.6521739130434783

In [63]:
mod = model_generator.create_model(seq_len = 512)
mod.save(path.join(DATA_DIR, "protein_bert/2022_03_07__08.pkl"))

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_07__08.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_07__08.pkl/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# Oversampling
# 3

In [30]:
wandb.init(project=f"March_finetune", entity="kvetab")

epoch,▁▁▂▂▃▃▄▄▅▅▁▂▂▃▃▄▁▁▂▂▃▃▄▄▅▅▆▆▇▇██▁▂▂▃▃▃▄▁
loss,█▄▄▃▃▃▃▃▃▃▃▂▂▂▂▁█▅▄▄▃▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▂
lr,██████████▁▁▁▁▁▁████████████▃▃▁▁▁▁▁▁▁▁▁▁
val_loss,▆▆▄▄▄▃▃▃▃▄▂▂▂▂▃▁█▆▅▄▄▃▃▃▃▄▂▃▂▃▂▂▂▃▂▁▁▂▁▁
best_epoch,9
best_val_loss,0.36406
epoch,0
loss,0.32212
lr,1e-05
val_loss,0.37174


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [31]:
sampler = RandomOverSampler(random_state=42)
x_train, y_train = sampler.fit_resample(train_data.drop("Y", axis=1), train_data['Y'])
x_valid, y_valid = sampler.fit_resample(valid_data.drop("Y", axis=1), valid_data['Y'])

In [32]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)



In [33]:
epoch_num = 100
batch_size = 128
learning_rate = 1e-4
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }
finetune(model_generator, input_encoder, OUTPUT_SPEC, x_train["seq"], y_train, x_valid['seq'], y_valid, \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)

[2022_03_07-12:59:26] Training set: Filtered out 0 of 2114 (0.0%) records of lengths exceeding 510.
[2022_03_07-12:59:26] Validation set: Filtered out 0 of 188 (0.0%) records of lengths exceeding 510.
[2022_03_07-12:59:26] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
17/17 [==============================] - 11s 228ms/step - loss: 0.8073 - val_loss: 0.7109
Epoch 2/100
17/17 [==============================] - 2s 110ms/step - loss: 0.6141 - val_loss: 0.6363
Epoch 3/100
17/17 [==============================] - 2s 109ms/step - loss: 0.5803 - val_loss: 0.5773
Epoch 4/100
17/17 [==============================] - 2s 109ms/step - loss: 0.5528 - val_loss: 0.5728
Epoch 5/100
17/17 [==============================] - 2s 109ms/step - loss: 0.5507 - val_loss: 0.5749
Epoch 6/100
17/17 [==============================] - 2s 110ms/step - loss: 0.5257 - val_loss: 0.5750
Epoch 7/100
17/17 [==============================] - 2s 110ms/step - loss: 0.5299 - val_loss: 0.5670
Epoch 8/100
17/17 [==============================] - 2s 109ms/step - loss: 0.5130 - val_loss: 0.5943
Epoch 9/100
17/17 [==============================] - 2s 109ms/step - loss: 0.5069 - val_loss: 0.5351
Epoch 10/100
17/17 [==============================] - 2s 109ms/step - loss: 0.4862 - val_l

17/17 [==============================] - 12s 367ms/step - loss: 0.4657 - val_loss: 0.5102
Epoch 2/100
17/17 [==============================] - 4s 258ms/step - loss: 0.4443 - val_loss: 0.5507
Epoch 3/100
17/17 [==============================] - 4s 259ms/step - loss: 0.4225 - val_loss: 0.4919
Epoch 4/100
17/17 [==============================] - 4s 259ms/step - loss: 0.4076 - val_loss: 0.5101
Epoch 5/100
17/17 [==============================] - 4s 259ms/step - loss: 0.4103 - val_loss: 0.5304
Epoch 6/100
17/17 [==============================] - 4s 259ms/step - loss: 0.3643 - val_loss: 0.4958

Epoch 00006: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
Epoch 7/100
17/17 [==============================] - 4s 259ms/step - loss: 0.3532 - val_loss: 0.4965
Epoch 8/100
17/17 [==============================] - 4s 258ms/step - loss: 0.3483 - val_loss: 0.4983
Epoch 9/100
17/17 [==============================] - 4s 259ms/step - loss: 0.3399 - val_loss: 0.5005

Epoch 00009: ReduceL

34/34 [==============================] - 16s 301ms/step - loss: 0.4216 - val_loss: 0.5015


In [34]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)
print('Confusion matrix:')
display(confusion_matrix)
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

Confusion matrix:


,0,1
0,75,21
1,1,22


0.6666666666666666

In [35]:
mod = model_generator.create_model(seq_len = 512)
mod.save(path.join(DATA_DIR, "protein_bert/2022_03_07__03.pkl"))

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_07__03.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_07__03.pkl/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# 9

In [64]:
wandb.init(project=f"March_finetune", entity="kvetab")

epoch,▁▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██▁▁▂▂▂▂▃▃▃▄▄▄▄▁
loss,█▅▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂
lr,█████████████▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▃▃▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁
best_epoch,12
best_val_loss,0.36563
epoch,0
loss,0.32678
lr,1e-05
val_loss,0.37264


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [65]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

In [66]:
training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 3, factor = 0.25, min_lr = 1e-07, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 6, restore_best_weights = True),
    WandbCallback()
]

In [67]:
epoch_num = 100
batch_size = 128
learning_rate = 1e-4
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }
finetune(model_generator, input_encoder, OUTPUT_SPEC, x_train["seq"], y_train, x_valid['seq'], y_valid, \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)

[2022_03_07-13:37:58] Training set: Filtered out 0 of 2114 (0.0%) records of lengths exceeding 510.
[2022_03_07-13:37:58] Validation set: Filtered out 0 of 188 (0.0%) records of lengths exceeding 510.
[2022_03_07-13:37:58] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
17/17 [==============================] - 11s 223ms/step - loss: 0.7751 - val_loss: 0.6406


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/100
17/17 [==============================] - 2s 108ms/step - loss: 0.5977 - val_loss: 0.6238
Epoch 3/100
17/17 [==============================] - 2s 108ms/step - loss: 0.5741 - val_loss: 0.5723
Epoch 4/100
17/17 [==============================] - 2s 109ms/step - loss: 0.5726 - val_loss: 0.6539
Epoch 5/100
17/17 [==============================] - 2s 109ms/step - loss: 0.5250 - val_loss: 0.5448
Epoch 6/100
17/17 [==============================] - 2s 110ms/step - loss: 0.5171 - val_loss: 0.5468
Epoch 7/100
17/17 [==============================] - 2s 109ms/step - loss: 0.4923 - val_loss: 0.5443
Epoch 8/100
17/17 [==============================] - 2s 109ms/step - loss: 0.4926 - val_loss: 0.5471
Epoch 9/100
17/17 [==============================] - 2s 109ms/step - loss: 0.4977 - val_loss: 0.5363
Epoch 10/100
17/17 [==============================] - 2s 109ms/step - loss: 0.4853 - val_loss: 0.5426
Epoch 11/100
17/17 [==============================] - 2s 109ms/step - loss: 0.4822 - val_l

17/17 [==============================] - 12s 370ms/step - loss: 0.4499 - val_loss: 0.5293
Epoch 2/100
17/17 [==============================] - 4s 258ms/step - loss: 0.4416 - val_loss: 0.5522
Epoch 3/100
17/17 [==============================] - 4s 259ms/step - loss: 0.4483 - val_loss: 0.6439
Epoch 4/100
17/17 [==============================] - 4s 258ms/step - loss: 0.4372 - val_loss: 0.4974
Epoch 5/100
17/17 [==============================] - 4s 258ms/step - loss: 0.4028 - val_loss: 0.4906
Epoch 6/100
17/17 [==============================] - 4s 259ms/step - loss: 0.3774 - val_loss: 0.4947
Epoch 7/100
17/17 [==============================] - 4s 259ms/step - loss: 0.3641 - val_loss: 0.4910
Epoch 8/100
17/17 [==============================] - 4s 259ms/step - loss: 0.3407 - val_loss: 0.4917

Epoch 00008: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
Epoch 9/100
17/17 [==============================] - 4s 259ms/step - loss: 0.3216 - val_loss: 0.5227
Epoch 10/100
17/17 [=

34/34 [==============================] - 16s 318ms/step - loss: 0.4017 - val_loss: 0.5252


In [68]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)
print('Confusion matrix:')
display(confusion_matrix)
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

Confusion matrix:


,0,1
0,78,18
1,1,22


0.6984126984126984

In [69]:
mod = model_generator.create_model(seq_len = 512)
mod.save(path.join(DATA_DIR, "protein_bert/2022_03_07__09.pkl"))

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_07__09.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_07__09.pkl/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# 4

In [36]:
sampler = RandomUnderSampler(random_state=42)
x_train_u, y_train_u = sampler.fit_resample(train_data.drop("Y", axis=1), train_data['Y'])

In [37]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

wandb.init(project=f"March_finetune", entity="kvetab")


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██▁▁▂▂▂▂▃▃▃▁
loss,█▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▃▃▃▂▂▂▁▁▁▁▂
lr,███████████████▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▄▄▄▄▃▄▂▃▅▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▃▁▂▂▁▁▁▁▁
epoch,0
loss,0.4216
lr,1e-05
val_loss,0.50154


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [38]:
epoch_num = 100
batch_size = 128
learning_rate = 5e-4
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }
finetune(model_generator, input_encoder, OUTPUT_SPEC, x_train_u["seq"], y_train_u, valid_data['seq'], valid_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)

[2022_03_07-13:07:54] Training set: Filtered out 0 of 562 (0.0%) records of lengths exceeding 510.
[2022_03_07-13:07:54] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_03_07-13:07:54] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
5/5 [==============================] - 10s 543ms/step - loss: 1.2386 - val_loss: 1.6062
Epoch 2/100
5/5 [==============================] - 1s 123ms/step - loss: 1.0543 - val_loss: 0.5909
Epoch 3/100
5/5 [==============================] - 1s 120ms/step - loss: 0.8385 - val_loss: 0.7112
Epoch 4/100
5/5 [==============================] - 1s 118ms/step - loss: 0.8166 - val_loss: 0.7024
Epoch 5/100
5/5 [==============================] - 1s 120ms/step - loss: 0.7512 - val_loss: 0.5992

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 6/100
5/5 [==============================] - 1s 120ms/step - loss: 0.6350 - val_loss: 0.5411
Epoch 7/100
5/5 [==============================] - 1s 118ms/step - loss: 0.6238 - val_loss: 0.7253
Epoch 8/100
5/5 [==============================] - 1s 118ms/step - loss: 0.6080 - val_loss: 0.7608
Epoch 9/100
5/5 [==============================] - 1s 119ms/step - loss: 0.5872 - val_loss: 0.6103

Epoch 00009: ReduceLROnPla

5/5 [==============================] - 1s 248ms/step - loss: 0.6278 - val_loss: 0.7628
Epoch 3/100
5/5 [==============================] - 1s 247ms/step - loss: 0.6260 - val_loss: 0.5725
Epoch 4/100
5/5 [==============================] - 1s 245ms/step - loss: 0.5982 - val_loss: 0.6099

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 5/100
5/5 [==============================] - 1s 245ms/step - loss: 0.5832 - val_loss: 0.6256
Epoch 6/100
5/5 [==============================] - 1s 245ms/step - loss: 0.5755 - val_loss: 0.6147
Epoch 7/100
5/5 [==============================] - 1s 245ms/step - loss: 0.5628 - val_loss: 0.6059

Epoch 00007: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
[2022_03_07-13:08:38] Training on final epochs of sequence length 1024...
[2022_03_07-13:08:38] Training set: Filtered out 0 of 562 (0.0%) records of lengths exceeding 1022.
[2022_03_07-13:08:38] Validation set: Filtered out 0 of 120 (0.0%) records of leng

9/9 [==============================] - 10s 485ms/step - loss: 0.6393 - val_loss: 0.6304


In [39]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)
print('Confusion matrix:')
display(confusion_matrix)
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

Confusion matrix:


,0,1
0,71,25
1,5,18


0.5454545454545454

In [40]:
# protein_bert/2022_03_07__04.pkl
mod = model_generator.create_model(seq_len = 512)
mod.save(path.join(DATA_DIR, "protein_bert/2022_03_07__04.pkl"))

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_07__04.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_07__04.pkl/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# Load Model

In [20]:
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

In [21]:
seq_len = 512
encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')

[2022_03_30-15:06:19] Test set: Filtered out 0 of 119 (0.0%) records of lengths exceeding 510.


In [22]:
test_X, test_Y, test_sample_weigths = encoded_test_set

In [23]:
model_path = path.join(DATA_DIR, "protein_bert/2022_03_30__05.pkl")
model = keras.models.load_model(model_path)

In [24]:
y_pred = model.predict(test_X, batch_size=32)
y_pred_classes = (y_pred >= 0.5)
f1_score(test_Y, y_pred_classes)

0.6486486486486486

In [25]:
tap_data = pd.read_csv(path.join(DATA_DIR, "tap/TAP_data.csv"))
tap_data.head()

,Antibody_ID,heavy,light,CDR_length,PSH,PPC,PNC,SFvCSP,Y
0,Abagovomab,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLD...,DIELTQSPASLSASVGETVTITCQASENIYSYLAWHQQKQGKSPQL...,46,129.7603,0.0000,0.0000,16.32,1
1,Abituzumab,QVQLQQSGGELAKPGASVKVSCKASGYTFSSFWMHWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDISNYLAWYQQKPGKAPKL...,45,115.9106,0.0954,0.0421,-3.10,1
2,Abrilumab,QVQLVQSGAEVKKPGASVKVSCKVSGYTLSDLSIHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQQKPGKAPKL...,45,109.6995,0.0000,0.8965,-4.00,1
3,Actoxumab,QVQLVESGGGVVQPGRSLRLSCAASGFSFSNYGMHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQHKPGKAPKL...,49,112.6290,0.0000,1.1247,3.10,1
4,Adalimumab,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQGIRNYLAWYQQKPGKAPKL...,48,111.2512,0.0485,1.1364,-19.50,1


In [26]:
tap_data["seq"] = tap_data["heavy"] +  tap_data["light"]

In [27]:
encoded_tap_set = encode_dataset(tap_data["seq"], tap_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'TAP set')

[2022_03_30-15:06:39] TAP set: Filtered out 0 of 241 (0.0%) records of lengths exceeding 510.


In [28]:
tap_X, tap_Y, tap_sample_weigths = encoded_tap_settap_X, tap_Y, tap_sample_weigths = encoded_tap_set

In [29]:
y_pred = model.predict(tap_X, batch_size=32)
y_pred_classes = (y_pred >= 0.5)
f1_score(tap_Y, y_pred_classes)

0.34285714285714286

In [30]:
accuracy_score(tap_Y, y_pred_classes)

0.23651452282157676

In [34]:
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
seq_len = 512
encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')
test_X, test_Y, test_sample_weigths = encoded_test_set

[2022_03_10-10:20:09] Test set: Filtered out 0 of 119 (0.0%) records of lengths exceeding 510.


In [40]:
def test_model(model_name):
    model_path = path.join(DATA_DIR, f"protein_bert/{model_name}")
    model = keras.models.load_model(model_path)
    y_pred = model.predict(test_X, batch_size=32)
    y_pred_classes = (y_pred >= 0.5)
    f1 = f1_score(test_Y, y_pred_classes)
    print(f"Model {model_name}")
    print(f"Test F1: {f1}")
    
    y_pred = model.predict(tap_X, batch_size=32)
    y_pred_classes = (y_pred >= 0.5)
    f1 = f1_score(tap_Y, y_pred_classes)
    acc = accuracy_score(tap_Y, y_pred_classes)
    print(f"TAP F1: {f1}")
    print(f"TAP Acc: {acc}")

In [36]:
test_model(f"2022_03_10__09_78.pkl")

Model 2022_03_10__09_78.pkl
Test F1: 0.6666666666666666
TAP F1: 0.7774936061381073
TAP Acc: 0.6390041493775933


In [39]:
for seed in seeds:
    test_model(f"2022_03_10__09_{seed:02d}.pkl")

Model 2022_03_10__09_02.pkl
Test F1: 0.7333333333333334
TAP F1: 0.7277628032345014
TAP Acc: 0.5809128630705395


Model 2022_03_10__09_13.pkl
Test F1: 0.7301587301587301
TAP F1: 0.7374005305039788
TAP Acc: 0.5892116182572614


Model 2022_03_10__09_19.pkl
Test F1: 0.7301587301587301
TAP F1: 0.7413333333333334
TAP Acc: 0.5975103734439834


Model 2022_03_10__09_27.pkl
Test F1: 0.7457627118644068
TAP F1: 0.7119565217391305
TAP Acc: 0.5601659751037344


Model 2022_03_10__09_38.pkl
Test F1: 0.6571428571428571
TAP F1: 0.7552083333333333
TAP Acc: 0.6099585062240664


Model 2022_03_10__09_42.pkl
Test F1: 0.6999999999999998
TAP F1: 0.688888888888889
TAP Acc: 0.5352697095435685


Model 2022_03_10__09_56.pkl
Test F1: 0.6666666666666666
TAP F1: 0.805
TAP Acc: 0.6763485477178424


Model 2022_03_10__09_63.pkl
Test F1: 0.7096774193548387
TAP F1: 0.7119565217391305
TAP Acc: 0.5601659751037344


Model 2022_03_10__09_06.pkl
Test F1: 0.7931034482758621
TAP F1: 0.6704225352112676
TAP Acc: 0.5145228215767634

In [44]:
for i in range(9):
    if i == 4:
        continue
    test_model(f"2022_03_07__{i + 1:02d}")

Model 2022_03_07__01
Test F1: 0.6666666666666666
TAP F1: 0.5345911949685533
TAP Acc: 0.38589211618257263
Model 2022_03_07__02
Test F1: 0.6923076923076923
TAP F1: 0.5970149253731343
TAP Acc: 0.43983402489626555
Model 2022_03_07__03
Test F1: 0.6666666666666667
TAP F1: 0.7552083333333333
TAP Acc: 0.6099585062240664
Model 2022_03_07__04
Test F1: 0.5454545454545454
TAP F1: 0.703601108033241
TAP Acc: 0.5560165975103735
Model 2022_03_07__06
Test F1: 0.5945945945945946
TAP F1: 0.3604240282685512
TAP Acc: 0.24896265560165975
Model 2022_03_07__07
Test F1: 0.5454545454545454
TAP F1: 0.3237410071942446
TAP Acc: 0.21991701244813278
Model 2022_03_07__08
Test F1: 0.6521739130434783
TAP F1: 0.529968454258675
TAP Acc: 0.3817427385892116
Model 2022_03_07__09
Test F1: 0.6984126984126985
TAP F1: 0.7154471544715447
TAP Acc: 0.5643153526970954


wandb: Network error (ReadTimeout), entering retry loop.


# Cross-validation

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
seeds = [2, 13, 19, 27, 38, 42, 56, 63, 6, 78]

In [11]:
def read_data_for_seed(seed):
    chen_train = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_train_{seed}.csv"), index_col=0)
    chen_test = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_test_{seed}.csv"), index_col=0)
    chen_valid, chen_test = train_test_split(chen_test, test_size=0.5, random_state=3)
    return chen_train, chen_valid, chen_test

In [13]:
wandb.init(project=f"Cross-val", entity="kvetab")
training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 3, factor = 0.25, min_lr = 1e-07, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 6, restore_best_weights = True),
    WandbCallback()
]

wandb: Currently logged in as: kvetab (use `wandb login --relogin` to force relogin)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [14]:
epoch_num = 100
batch_size = 128
learning_rate = 1e-4
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }

In [15]:
def train_on_split(seed):
    train, valid, test = read_data_for_seed(seed)
    train["seq"] = train["heavy"] + train["light"]
    valid["seq"] = valid["heavy"] + valid["light"]
    test["seq"] = test["heavy"] + test["light"]
    sampler = RandomOverSampler(random_state=42)
    x_train, y_train = sampler.fit_resample(train.drop("Y", axis=1), train['Y'])
    x_valid, y_valid = sampler.fit_resample(valid.drop("Y", axis=1), valid['Y'])
    wandb.init(project=f"Cross-val", entity="kvetab")
    model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)
    
    finetune(model_generator, input_encoder, OUTPUT_SPEC, x_train["seq"], y_train, x_valid['seq'], y_valid, \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)
    
    results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test['seq'], test['Y'], \
            start_seq_len = 512, start_batch_size = 32)
    print(f"Training split {seed}")
    print('Confusion matrix:')
    display(confusion_matrix)
    fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
    f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
    print(f1)
    
    mod = model_generator.create_model(seq_len = 512)
    mod.save(path.join(DATA_DIR, f"protein_bert/2022_03_10__09_{seed:02d}.pkl"))

In [16]:
for s in seeds:
    train_on_split(s)

wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


[2022_03_10-09:31:24] Training set: Filtered out 0 of 2002 (0.0%) records of lengths exceeding 510.
[2022_03_10-09:31:24] Validation set: Filtered out 0 of 246 (0.0%) records of lengths exceeding 510.
[2022_03_10-09:31:24] Training with frozen pretrained layers...


2022-03-10 09:31:25.040869: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-10 09:31:25.598857: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9656 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:21:00.0, compute capability: 7.5
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-03-10 09:31:27.522882: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are en

Epoch 1/100


2022-03-10 09:31:35.358668: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 7605


...

,0,1
0,75,64
1,6,25


0.4166666666666667


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_10__09_78.pkl/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_03_10__09_78.pkl/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
